In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1732412967921_0001,pyspark,idle,Link,Link,None,


In [2]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1732412967921_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1732412967921_0002/container_1732412967921_0002_01_000001/tmp/spark-4f3b8193-5960-417f-92f5-f5191ae2f8e8
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.15.30 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [3]:
from pyspark.sql.functions import expr, col, udf, desc, collect_list
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, FloatType
from pyspark.sql import functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark.sql import Window
from pyspark.sql.functions import row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.csv("s3://proyecto-mineria-de-datos/user-filtered.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df = df.filter(df['rating'] != 0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

In [8]:
df = df.orderBy("user_id", "anime_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      21|    10|
|      0|      24|     9|
|      0|      67|     9|
|      0|      68|     6|
|      0|     121|     9|
|      0|     164|     8|
|      0|     169|     7|
|      0|     174|     4|
|      0|     199|     8|
|      0|     235|    10|
|      0|     242|    10|
|      0|     269|     9|
|      0|     356|     9|
|      0|     415|    10|
|      0|     419|     8|
|      0|     430|     9|
|      0|     431|     8|
|      0|     433|     6|
|      0|     448|     5|
|      0|     459|     9|
+-------+--------+------+
only showing top 20 rows

In [10]:
df_final = df

total_users = df.select("user_id").distinct().count()
total_animes = df.select("anime_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
indexer_user = StringIndexer(inputCol='user_id', outputCol='user_index').setHandleInvalid("keep")
indexer_anime = StringIndexer(inputCol='anime_id', outputCol='anime_index').setHandleInvalid("keep")

user_indexer_model = indexer_user.fit(df_final)
anime_indexer_model = indexer_anime.fit(df_final)

df_final = user_indexer_model.transform(df_final)
df_final = anime_indexer_model.transform(df_final)

df_final = df_final.withColumn('user_index', df_final['user_index'].cast('integer'))
df_final = df_final.withColumn('anime_index', df_final['anime_index'].cast('integer'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
percent_users_to_mask = 0.95
percent_animes_to_mask = 0.9

user_cutoff = int(total_users * (1 - percent_users_to_mask))
anime_cutoff = int(total_animes * (1 - percent_animes_to_mask))
print(user_cutoff, total_users)
print(anime_cutoff, total_animes)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15683 313670
1717 17172

In [13]:
train_data = df_final.filter(~((col("user_index") > user_cutoff) & (col("anime_index") > anime_cutoff)))
test_data = df_final.filter((col("user_index") > user_cutoff) & (col("anime_index") > anime_cutoff))
print(train_data.count())
print(test_data.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53656724
8740988

In [14]:
final_als = ALS(
    userCol='user_index',
    itemCol='anime_index',
    ratingCol='rating',
    coldStartStrategy='drop',
    nonnegative=True,
    rank=20,
    maxIter=20,
    regParam=0.15
)

best_model = final_als.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
predictions = best_model.transform(test_data)
predictions = predictions.withColumn("prediction", expr("CASE WHEN prediction < 1 THEN 1 WHEN prediction > 10 THEN 10 ELSE prediction END"))
evaluator = RegressionEvaluator(labelCol='rating', predictionCol='prediction')

metrics = {}

metric_names = ['rmse', 'mae', 'mse', 'r2']

for metric in metric_names:
    evaluator.setMetricName(metric)
    metrics[metric] = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {metrics['rmse']}")
print(f"Mean Absolute Error (MAE): {metrics['mae']}")
print(f"Mean Squared Error (MSE): {metrics['mse']}")
print(f"R² (coefficient of determination): {metrics['r2']}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE): 1.3907441978772679
Mean Absolute Error (MAE): 1.0616008498466918
Mean Squared Error (MSE): 1.9341694239292855
R? (coefficient of determination): 0.4282145764920152

In [21]:
def evaluate_als_recommendations(model, test_data, k=10):
    user_subset = test_data.select('user_index').distinct()
    recommendations = model.recommendForUserSubset(user_subset, k)

    predicted_rankings = recommendations.rdd.map(
        lambda row: (row.user_index, [int(x.anime_index) for x in row.recommendations])
    )

    actual_rankings = test_data.groupBy('user_index').agg(
        F.collect_list('anime_index').alias('actual_items')
    ).rdd.map(
        lambda row: (row.user_index, [int(x) for x in row.actual_items])
    )

    prediction_truth = predicted_rankings.join(actual_rankings)

    metrics = RankingMetrics(prediction_truth.map(lambda x: (x[1][0], x[1][1])))

    # Calculate various metrics
    results = {
        'MAP': metrics.meanAveragePrecision,
        'NDCG at K': metrics.ndcgAt(k),
        'Precision at K': metrics.precisionAt(k),
        'Recall at K': metrics.recallAt(k)
    }

    return results

# Use the evaluation function
metrics_results = evaluate_als_recommendations(best_model, test_data, k=10)

# Print results
for metric_name, value in metrics_results.items():
    print(f"{metric_name}: {value:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MAP: 0.0002
NDCG at K: 0.0016
Precision at K: 0.0018
Recall at K: 0.0007
/mnt/yarn/usercache/livy/appcache/application_1732412967921_0002/container_1732412967921_0002_01_000001/pyspark.zip/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [17]:
final_als = ALS(
    userCol='user_index',
    itemCol='anime_index',
    ratingCol='rating',
    coldStartStrategy='drop',
    nonnegative=True,
    rank=10,
    maxIter=20,
    regParam=0.15
)

final_model = final_als.fit(df_final)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
user_converter = IndexToString(inputCol='user_index', outputCol='user_id')
anime_converter = IndexToString(inputCol='anime_index', outputCol='anime_id')

user_converter.setLabels(user_indexer_model.labels)
anime_converter.setLabels(anime_indexer_model.labels)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

IndexToString_6fbf3eb24a45

In [23]:
userRecs = final_model.recommendForAllUsers(10)
userRecs = user_converter.transform(userRecs)

def create_reverse_recommendations():
    anime_labels = anime_indexer_model.labels

    @udf(ArrayType(StructType([
        StructField("anime_id", StringType(), True),
        StructField("rating", FloatType(), True)
    ])))
    def convert_recommendations(recommendations):
        return [{"anime_id": str(anime_labels[int(rec.anime_index)]),
                "rating": float(rec.rating)}
                for rec in recommendations]

    return convert_recommendations

convert_recs = create_reverse_recommendations()
userRecs = userRecs.withColumn("recommendations", convert_recs(col("recommendations")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
def format_recommendations_for_export(userRecs):
    flattened = userRecs.select(
        'user_id',
        F.posexplode('recommendations').alias('position', 'rec')
    ).select(
        'user_id',
        'position',
        'rec.anime_id',
        'rec.rating'
    )

    pivoted = flattened.groupBy('user_id').pivot('position').agg(
        F.first('anime_id').alias('anime_id'),
        F.first('rating').alias('predicted_rating')
    )

    for i in range(10):
        pivoted = (pivoted
            .withColumnRenamed(f'{i}anime_id', f'recommendation_{i+1}_anime_id')
            .withColumnRenamed(f'{i}_predicted_rating', f'recommendation_{i+1}_score'))

    final_df = pivoted.orderBy('user_id')

    return final_df

formatted_recs = format_recommendations_for_export(userRecs)

formatted_recs.toPandas().to_csv('anime_recommendations.csv', index=False)

formatted_recs.coalesce(1).write.mode('overwrite').option('header', 'true').csv('s3://proyecto-mineria-de-datos/anime_recommendations_spark')

formatted_recs.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+-----------------------+
|user_id|0_anime_id|recommendation_1_score|1_anime_id|recommendation_2_score|2_anime_id|recommendation_3_score|3_anime_id|recommendation_4_score|4_anime_id|recommendation_5_score|5_anime_id|recommendation_6_score|6_anime_id|recommendation_7_score|7_anime_id|recommendation_8_score|8_anime_id|recommendation_9_score|9_anime_id|recommendation_10_score|
+-------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+----------------------+----------+------

In [25]:
final_model.write().overwrite().save("s3://proyecto-mineria-de-datos/anime_recommendations_als_model")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…